ELEC-E5550 - Statistical Natural Language Processing
# SET 5: Morphological segmentation

# Released: 12.02.2020
# Deadline: 25.03.2020 at midnight

After completing this assignment, you'll learn how to tokenize text into subwords using BPE algorithm

KEYWORDS:
* subwords
* BPE

We've already talked about the problem of segmenting text into appropriate units (tokenization). Back then, we were considering words as those units, but there are other units that you should probably explore as well: characters and **subwords**. In this assignment, we're going to focus on **tokenization into subwords**.

The motivation to segmenting words further into smaller elements comes from morphology, where such elements are called **morphemes**. A **morpheme** is defined as the smallest meaning-bearing unit of a language. For example, the word *unpredictable* contains three morphemes: *un*, *predict* and *able*. As you can see, morphemes are not unique to one word, they are elements that are regularly seen in other words too. For example, *un* in *unhappy*, *predict* in *predictive*, and *able* in *comfortable*. Thus, just as sentences are constructed from words, words are constructed from morphemes. 

Segmenting into morphemes (especially in languages with rich morphology) helps to avoid the problem of out-of-vocabulary (OOV) words in text corpora. For example, if our training corpus contains *cool*, *cool-est* and *dumb-er* when the new word *cooler* comes in, it can be analyzed into *cool-er*.

## TASK 1
# BPE
One approach to tokenization into subwords is based on the **byte pair encoding** (**BPE**) algorithm for text compression. It iteratively merges frequent pairs of characters forming new subwords. The intuition here is that morphemes are frequently repeated substrings, so this method should merge symbols into them instead of into some random meaningless character sequences. The algorithm is applied only inside words (there is no merges across word boundaries). 

**BPE** algorithm begins with its vocabulary being a set of characters seen in the training corpus. Each word in the corpus is represented as a sequence of characters plus a special end-of-word symbol '_'. At each iteration step $k_i$, the algorithm counts the number of symbol pairs, finds the most frequent pair ['A','B'] and replaces it with the new merged symbol ‘AB’. The algorithm stops when it's done $k$ merges ($k$ is a parameter of the algorithm). The algorithm begins with the set of symbols equal to the set of characters. The resulting symbol set should have the original set of characters plus $k$ new symbols. 

To learn segmentations with **BPE**, you should take the following steps:
* STEP 1: tokenize a training corpus into words and collect frequency statistics of word tokens in the training corpus.
* STEP 2: represent each word as a list of characters plus a special end-of-word symbol '_'.
* STEP 3: count the frequencies of symbol pairs.
* STEP 4: replace every occurance of the most frequent pair ['A','B'] with the new merged symbol 'AB'.
* STEP 5: repeat STEPs 3-4 k-1 times more.

To segment a test corpus with learned segmentations, you shoud:
* STEP 6: tokenize a test corpus into words (with the same tokenisation algorith as in trainig).
* STEP 7: represent each word as a list of characters plus a special end-of-word symbol '_'.
* STEP 8: for every word apply each merge operation in the order they were learned.

### STEP 1
### Tokenize into words and collect frequencies
## 1.1
Write a function that reads a text from a file, tokenizes it into words by whitespaces and collects the word frequencies

In [1]:
from collections import Counter

def collect_word_counts(file_name):
    """
    this function takes in a path to a text file, reads the file,
    tokenizes it into words, and then counts their frequencies.
    
    INPUT:
    file_name - a path to a training corpus as a string
    OUTPUT:
    word_counts - a frequency dictionary of words
    """
    
    # YOUR CODE HERE
    word_counts = {}

    file = open(file_name, 'r').read().split()

    for word in file:
        if word in word_counts.keys():
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    
    return word_counts

In [2]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_corpus_path = "/coursedata/SUBS/dummy_corpus.txt"

# check that the output of the function is a list
assert_equal(type(collect_word_counts(dummy_corpus_path)), dict)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
assert_equal(collect_word_counts(dummy_corpus_path)['low'], 5)
assert_equal(collect_word_counts(dummy_corpus_path)['lowest'], 2)
assert_equal(collect_word_counts(dummy_corpus_path)['new'], 2)
assert_equal(collect_word_counts(dummy_corpus_path)['newer'], 6)
assert_equal(collect_word_counts(dummy_corpus_path)['wider'], 3)


### STEP 2
### Convert words into strings of characters
## 1.2
Now, represent each word in your frequency dictionary as a tuple of characters plus a special end-of-word symbol '_'.

In [3]:
def convert_to_chars(vocab):
    """
    this function takes in a frequeny dictionary of words in the trainin corpus, 
    and converts the key words to a tuple of characters plus a special end-of-word symbol '_'.
    
    INPUT:
        vocab - a frequency dictionary of words
    OUTPUT:
        separated_vocab - a frequency dictionary of words 
        represented as a tuple of characters plus a special end-of-word symbol '_'
    """
    # YOUR CODE HERE
    separated_vocab = {}
    for key in vocab.keys():
        separated_vocab[tuple(key + '_')] = vocab[key]
    return separated_vocab

In [4]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_freq_vocab = {'low': 5, 'lowest': 2, 'newer': 6, 'wider': 3, 'new': 2}

# check that the output of the function is a list
assert_equal(type(convert_to_chars(dummy_freq_vocab)), dict)
# check that the keys are tuples
assert_equal(type(list(convert_to_chars(dummy_freq_vocab).keys())[0]), tuple)
#check that there are no new elements
assert_equal(len(convert_to_chars(dummy_freq_vocab)), 5)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD

assert_equal(convert_to_chars(dummy_freq_vocab)[('l', 'o', 'w', '_')], 5)
assert_equal(convert_to_chars(dummy_freq_vocab)[('l', 'o', 'w', 'e', 's', 't', '_')], 2)
assert_equal(convert_to_chars(dummy_freq_vocab)[('n', 'e', 'w', 'e', 'r', '_')], 6)
assert_equal(convert_to_chars(dummy_freq_vocab)[('w', 'i', 'd', 'e', 'r', '_')], 3)
assert_equal(convert_to_chars(dummy_freq_vocab)[('n', 'e', 'w', '_')], 2)


### STEP 3
### Collect frequencies of symbol pairs
## 1.3
Write a function that takes in a frequency dictionary, where keys are words represented as tuples of symbols, and outputs the most frequent pair of symbols in the corpus. In the case, when there are several pairs with the same frequency, return the pair that is earlier alphabetically.

In [5]:
def get_the_pair_to_merge(vocab_as_symbols):
    """
    this function takes in a frequency dictionary, where keys are words represented as tuples of symbols, 
    and outputs the most frequent pair of symbols in the corpus
    
    INPUT:
        vocab_as_symbols - a frequency dictionary, where keys are words represented as tuples of symbols
    OUTPUT:
        merge_pair - the most frequent pair of symbols, a pair to merge (a tuple)
    """
    
    # YOUR CODE HERE
    pair_freq = {}
    for key in vocab_as_symbols.keys():
        for i in range(len(key) - 1):
            if key[i:i + 2] in pair_freq.keys():
                pair_freq[key[i:i + 2]] += vocab_as_symbols[key]
            else:
                pair_freq[key[i:i + 2]] = vocab_as_symbols[key]
    merge_pair = max(pair_freq.keys(), key=(lambda k: pair_freq[k]))
    return merge_pair

In [6]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_freq_vocab_as_symbols = {('l', 'o', 'w', '_'): 5,
                               ('l', 'o', 'w', 'e', 's', 't', '_'): 2,
                               ('n', 'e', 'w', 'e', 'r', '_'): 6, 
                               ('w', 'i', 'd', 'e', 'r', '_'): 3, 
                               ('n', 'e', 'w', '_'): 2}

# check that the output of the function is a tuple
assert_equal(type(get_the_pair_to_merge(dummy_freq_vocab_as_symbols)), tuple)
# check that the output of the function is a tuple of strings
assert_equal(type(get_the_pair_to_merge(dummy_freq_vocab_as_symbols)[0]), str)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD

assert_equal(get_the_pair_to_merge(dummy_freq_vocab_as_symbols), ('e', 'r'))


### STEP 4
### Merge the most frequent pair
## 1.4
Write a function that takes in a pair of symbols to merge and a frequency dictionary where words are represented as tuples of symbols, and returns a frequency dictionary, where words are still represented as tuples of symbols, but the most frequnt pair was merged in every word.

In [7]:
def merge(vocab_as_symbols, merge_pair):
    """
    this function takes in a frequency dictionary, where keys are words represented as tuples of symbols, 
    and outputs the most frequent pair of symbols in the corpus
    
    INPUT:
        merge_pair - a pair to merge (a tuple)
        vocab_as_symbols - a frequency dictionary, where keys are words represented as tuples of symbols
    OUTPUT:
        new_vocab_as_symbols - a frequency dictionary, where keys are words represented as tuples of symbols,
        with the given pair represented as a new symbol (concatenated pair)
    """
    # YOUR CODE HERE
    new_vocab = {}

    def find_sub_idx(test_list, repl_list, start=0):
        length = len(repl_list)
        for idx in range(start, len(test_list)):
            if test_list[idx: idx + length] == repl_list:
                return idx, idx + length

    def replace_sub(test_list, repl_list, new_list):
        length = len(new_list)
        idx = 0
        for start, end in iter(lambda: find_sub_idx(test_list, repl_list, idx), None):
            test_list[start: end] = new_list
            idx = start + length
        return test_list

    for key in vocab_as_symbols.keys():
        new_key = replace_sub(list(key), list(merge_pair), [''.join(merge_pair)])
        new_vocab[tuple(new_key)] = vocab_as_symbols[key]
    return new_vocab

In [8]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_vocab_as_symbols = {('l', 'o', 'w', '_'): 5,
                               ('l', 'o', 'w', 'e', 's', 't', '_'): 2,
                               ('n', 'e', 'w', 'e', 'r', '_'): 6, 
                               ('w', 'i', 'd', 'e', 'r', '_'): 3, 
                               ('n', 'e', 'w', '_'): 2}

dummy_pair = ('e', 'r')

# check that the output of the function is a dictionary
assert_equal(type(merge(dummy_vocab_as_symbols, dummy_pair)), dict)
# check that the keys are tuples
assert_equal(type(list(merge(dummy_vocab_as_symbols, dummy_pair).keys())[0]), tuple)
#check that there are no new elements
assert_equal(len(merge(dummy_vocab_as_symbols, dummy_pair)), 5)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
# check that the pair was merged everywhere
assert_equal(list(merge(dummy_vocab_as_symbols, dummy_pair).keys()), [('l', 'o', 'w', '_'), 
                                                                    ('l', 'o', 'w', 'e', 's', 't', '_'), 
                                                                    ('n', 'e', 'w', 'er', '_'), 
                                                                    ('w', 'i', 'd', 'er', '_'), 
                                                                    ('n', 'e', 'w', '_')])


### STEP 1-5
### Combine all functions and make all k merges
## 1.5
Now let's combine steps 1-5 into a function, that learns $k$ BPE merges.

In [9]:
def learn_BPE_merges(file_name, k):
    """
    this function takes in a frequency dictionary, where keys are words represented as tuples of symbols, 
    and outputs the most frequent pair of symbols in the corpus
    
    INPUT:
        file_name - a path to a training corpus as a string
        k - a number of merges to be learned
    OUTPUT:
        merges - a list of k merges in the order they were learned
        one merge is a tuple of two most frequent symbols at step k
    """
    
    merges = []
    # YOUR CODE HERE
    freq_dict = collect_word_counts(file_name)
    separated_vocab = convert_to_chars(freq_dict)

    for i in range(k):
        merges.append(get_the_pair_to_merge(separated_vocab))
        separated_vocab = merge(separated_vocab, merges[i])

    return merges

In [10]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_corpus_path = "/coursedata/SUBS/dummy_corpus.txt"


# check that the output of the function is a list
assert_equal(type(learn_BPE_merges(dummy_corpus_path, 10)), list)
# check that the output of the function is a list of tuples
assert_equal(type(learn_BPE_merges(dummy_corpus_path, 10)[0]), tuple)
# check that the output of the function is a list of tuples of strings
assert_equal(type(learn_BPE_merges(dummy_corpus_path, 10)[0][0]), str)
#check that there are exactly k merges
assert_equal(len(learn_BPE_merges(dummy_corpus_path, 10)), 10)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
# check that the pair was merged everywhere
assert_equal(learn_BPE_merges(dummy_corpus_path, 8), [('e', 'r'),
                                                     ('er', '_'),
                                                     ('e', 'w'),
                                                     ('n', 'ew'),
                                                     ('l', 'o'),
                                                     ('lo', 'w'),
                                                     ('new', 'er_'),
                                                     ('low', '_')])


AssertionError: Lists differ: [('e'[18 chars]), ('n', 'e'), ('ne', 'w'), ('l', 'o'), ('lo',[31 chars]'_')] != [('e'[18 chars]), ('e', 'w'), ('n', 'ew'), ('l', 'o'), ('lo',[31 chars]'_')]

First differing element 2:
('n', 'e')
('e', 'w')

  [('e', 'r'),
   ('er', '_'),
-  ('n', 'e'),
-  ('ne', 'w'),
?    -

+  ('e', 'w'),
+  ('n', 'ew'),
   ('l', 'o'),
   ('lo', 'w'),
   ('new', 'er_'),
   ('low', '_')]

### STEPS 6-8
### Segment a test corpus
## 1.6
Well, now we can apply what we've learned to segment any text. Write a function that reads the test corpus, and applies the merges we've learned. Note that you will probably need to adapt your previous functions. For example, we don't need to count the word frequencies anymore, since they don't play any role here.

Note: don't forget to get rid of the special end-of-word symbols.

In [11]:
def segment_text(file_name, merges):
    """
    this function takes in a path to a text file, reads the file,
    and tokenizes it into subwords in accorance with the learned BPE merges.
    
    INPUT:
        file_name - a path to a text as a string
        merges - a list of k merges in the order they were learned
        
    OUTPUT:
        segmented_text - a texted segmented with BPE. It's a list of words
        where each word is a string, with its segments separated by whitespaces
    """
    
    # YOUR CODE HERE
    file = open(file_name, 'r').read().split()
    vocab_dict = {key: key for key in file}
    vocab_as_symbols = convert_to_chars(vocab_dict)
    segmented_text = []
    for pair in merges:
        vocab_as_symbols = merge(vocab_as_symbols, pair)
    keys = list(vocab_as_symbols.keys())
    for i in range(len(keys)):
        seperator = ' '
        keys[i] = seperator.join(keys[i]).strip(' _')
    vals = list(vocab_as_symbols.values())
    for word in file:
        segmented_text.append(keys[vals.index(word)])
    return segmented_text

In [12]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_corpus_path = "/coursedata/SUBS/dummy_corpus.txt"

dummy_merges = [('e', 'r'),
                 ('er', '_'),
                 ('e', 'w'),
                 ('n', 'ew'),
                 ('l', 'o'),
                 ('lo', 'w')]


# check that the output of the function is a list
assert_equal(type(segment_text(dummy_corpus_path, dummy_merges)), list)
# check that the output of the function is a list of strings
assert_equal(type(segment_text(dummy_corpus_path, dummy_merges)[0]), str)


# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
# check that the train corpus is segmented the way it should:

assert_equal(segment_text(dummy_corpus_path, dummy_merges), ['low',
                                                             'low',
                                                             'low',
                                                             'low',
                                                             'low',
                                                             'low e s t',
                                                             'low e s t',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'w i d er',
                                                             'w i d er',
                                                             'w i d er',
                                                             'new',
                                                             'new'])


# check that the test corpus is segmented the way it should:
dummy_test_path = "/coursedata/SUBS/dummy_test_corpus.txt"
assert_equal(segment_text(dummy_test_path, dummy_merges), ['low er', 'c o o l er'])


## TASK 2
# ANALYZE SEGMENTATIONS
### Count word OOV
## 2.1
Now that we've done with the algorithm, let's see if it will actually help us with the OOV problem.
Let's use the same corpus as we did in the previous assignment (POS-tagging). We've randomly shuffled the sentences and split the corpus in roughly half. One half will be our training example, and another will be our test example.

Analyse:
1. The number of words in the vocabulry of the training corpus 
2. The number of words in the vocabulry of the test corpus
3. The number of words in the test corpus, that were not seen in the training corpus

Run the cell below, to collect the tokenizes corpora, type in the answer in the next cell.

In [13]:
f = open("/coursedata/SUBS/gum_train.txt", 'r')
train = f.read()
f.close()
train = train.split()

f = open("/coursedata/SUBS/gum_test.txt", 'r')
test = f.read()
f.close()
test = test.split()

In [14]:
# type in the answer as integer number
# For example:
# len_of_train_vocab = 234
len_of_train_vocab = 8677

# type in the answer as integer number
# For example:
# len_of_test_vocab = 234
len_of_test_vocab = 8770

# type in the answer as a number between 0 and 1
# For example:
# oov_portion = 0.5
oov_portion = 5035 / len_of_train_vocab

#Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE

In [15]:
### This cell contains hidden tests for the correct answers.
from numpy.testing import assert_almost_equal
from nose.tools import assert_equal


### Count subword OOV
## 2.2
Let's compare the OOV numbers we've got in the case where the text is tokenized by words and the case when it's tokenized by subwords. 

Learn 5000 BPE segmentation on the train data, then segment both corpora and compare the vocabulary numbers again.

In [ ]:
merges = learn_BPE_merges("/coursedata/SUBS/gum_train.txt", 5000)
segmented_train= segment_text("/coursedata/SUBS/gum_train.txt", merges)
segmented_test = segment_text("/coursedata/SUBS/gum_test.txt", merges)

In [ ]:
# type in the answer as integer number
# For example:
# len_of_train_sub_vocab = 234
len_of_train_sub_vocab = 4306

# type in the answer as integer number
# For example:
# len_of_test_sub_vocab = 234
len_of_test_sub_vocab = 3643

# type in the answer as a number between 0 and 1
# For example:
# oov_sub_portion = 0.5
oov_sub_portion = 137 / len_of_train_sub_vocab

#Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE

In [ ]:
### This cell contains hidden tests for the correct answers.
from numpy.testing import assert_almost_equal
from nose.tools import assert_equal


### Does the segmentation make sense?
## 2.3
Now let's look a bit closer at the subwords that we've learned.

1. What are the top 10 most frequent subwords in the test corpus? (in decending order)
2. What are the top 5 longest subwords in the test corpus?
3. What are the top 5 most frequent legths of subwords in the test corpus?

In [ ]:
# type in the answer as a list of strings
# For example:
# top_10_by_freq = ['a','b'...]
top_10_by_freq = ['e_', 's_', 'th', 't_', 'd_', 'in', 'an', 'er', 'y_', ',_']

# type in the answer as a list of strings
# For example:
# top_5_by_len = ['a','b'...]
top_5_by_len = ['incomprehensible_', 'representative_', 'discrimination_', 'transportation_', 'photographers_']

# type in the answer as a list of integers
# For example:
# top_5_freqs_of_lens= [1,2,3...]
top_5_freqs_of_lens = [3, 4, 5, 6, 7]

# Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE

In [ ]:
### This cell contains hidden tests for the correct answers.
from nose.tools import assert_equal


### Your thoughts
## 2.4
Briefly answer the following questions:

1. Describe what will happen if you change the k parameter? How to find a good number for k?

2. What are the possible NLP applications that can benefit from the tokenisation into subwords? 

3. How the OOV number for our data can be lowered further without changing anything in the segmentation procedure (k stays the same)


In [ ]:
# YOUR CODE HERE
